<a href="https://colab.research.google.com/github/lazregroudayna/RAG_Chatbot/blob/main/RAG_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch
!pip install -q accelerate  bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.4 MB/s eta 0:00:00


In [9]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.1 MB/s eta 0:00:00


In [10]:
!pip install langchain pypdf faiss-cpu google-search-results sentence-transformers langchainhub

In [11]:
from transformers import AutoTokenizer , AutoModelForCausalLM, BitsAndBytesConfig , pipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
import torch
import os

In [12]:
from getpass import getpass

In [16]:
token = getpass()

··········


In [17]:

os.environ["HF_TOKEN"] = token

In [20]:
model_name="mistralai/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [24]:
llm=pipeline(
    model=model,
    task="text-generation",
    tokenizer=tokenizer,
    temperature=0.3,
    return_full_text=True,
    do_sample=True,
    max_new_tokens=1000,
)

Device set to use cuda:0


In [25]:
mistral_llm = HuggingFacePipeline(pipeline=llm)

<ipython-input-25-2340dbab321e>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  mistral_llm = HuggingFacePipeline(pipeline=llm)


In [26]:
#doc = PyPDFLoader("/content/"+list(document.keys())[0])

In [27]:
#doc=doc.load()

In [28]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)

Function that takes the context and the question and generated an answer.

In [29]:
def getResponse(context, question):
  text=text_splitter.create_documents([context])
  db = FAISS.from_documents(text,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
  retriever = db.as_retriever()
  template="""[INST] You are helpful Question Answering Ai system, that try to answer user's questions depending on the given context .
In case you don't have the answer just say I don't have any informations.
You need to reply only for question so if the user's text doesn't seems like question type don't give him any informations .
Here the context you need :
{context}
##
Question : {text}
[/INST]"""

  prompt=PromptTemplate(
      input_variables=["text"],
      template=template
  )
  chain=LLMChain(prompt=prompt,llm=mistral_llm)
  rag_chain = (
  {"context": retriever, "text": RunnablePassthrough()}
      | chain
  )
  result=rag_chain.invoke(question)
  response_text=result['text']

  response_text = response_text.split("##")[1]
  final_response = response_text.split("[/INST]")[1]
  return final_response

In [30]:
getResponse("The ocean teems with an extraordinary diversity of life, from the tiniest plankton to the largest whales. Beneath the surface, a bustling ecosystem thrives, where intricate coral reefs provide shelter for vibrant fish and crustaceans. Deep-sea creatures, adapted to extreme pressure and darkness, roam mysterious abyssal plains. Majestic sea turtles gracefully navigate through the currents, while playful dolphins dart through the waves with unparalleled agility. Sharks, apex predators of the sea, command respect with their sleek power. Every corner of the ocean holds wonders waiting to be discovered, reminding us of the intricate interconnectedness and beauty of marine life.","What's the paragraph about")

<ipython-input-29-da5de4a6d231>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-29-da5de4a6d231>:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(prompt=prompt,llm=mistral_llm)


' The paragraphs in the context are about the wonders and beauty of the sea and its marine life. They describe the intricate interconnectedness and power of the ocean, as well as the unique creatures that inhabit it, particularly those found in the deep sea.'

In [66]:
!pip install PyPDF2



In [59]:
!ngrok config add-authtoken 2fpPlreRPSItBLHhZpAUdVmZfxO_36D2L6iihb8a6ppkYEvfG

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [60]:
!ngrock http 80


/bin/bash: line 1: ngrock: command not found


In [61]:
!pip install fastapi
!pip install pyngrok
!pip install uvicorn


In [62]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn


In [67]:
from fastapi.responses import JSONResponse
from PyPDF2 import PdfReader
import base64
import io

In [68]:
class Item(BaseModel):
    context: str
    question: str

class FileData(BaseModel):
    filename: str
    content: str

from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
    "http://172.25.96.1:5500",
    "http://localhost:5500/"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [69]:
@app.get('/')
async def root():
    return {'hello': 'Asslema fafa the api is working, Best Regards bobo\n you can use it by doing a post request on /ask with a payload of {context, question}'}


@app.post("/ask")
async def post_question(item: Item):
  result = getResponse(item.context, item.question)
  return {"Response": result}

@app.post("/upload")
async def upload_file(item:Item):
  try:
    result = getResponse(item.context, item.question)
    return {"result": result}
  except Exception as e:
    raise HTTPException(status_code=500, detail=str(e))

In [70]:
port = 2000
ngrok_tunnel = ngrok.connect(port)

In [71]:
ngrok_tunnel.public_url

'https://631e-34-125-183-119.ngrok-free.app'

In [72]:
nest_asyncio.apply()
uvicorn.run(app, port=port)

INFO:     Started server process [1468]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:2000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1468]
